# Knowledge Graph Construction

In [6]:
# !pip install neo4j

     ---------------------------------------- 0.0/196.5 kB ? eta -:--:--
     -------- ------------------------------ 41.0/196.5 kB 1.9 MB/s eta 0:00:01
     -------------------------------------  194.6/196.5 kB 2.4 MB/s eta 0:00:01
     -------------------------------------- 196.5/196.5 kB 2.0 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for neo4j: filename=neo4j-5.15.0-py3-none-any.whl size=272510 sha256=d0ea6ca159a8c73202942db1c70f8aaa95ea50ea1e948184d52e387997d917fd
  Stored in directory: c:\users\84166\appdata\local\pip\cache\wheels\b5\1c\7e\47445652c3ff85f3f13d094

In [7]:
import pandas as pd
from neo4j import GraphDatabase
import time

# Connect to the Neo4j database
uri = "bolt://localhost:7687"  
username = "neo4j"  
password = "00330000"

# Create a Neo4j database connection driver
driver = GraphDatabase.driver(uri, auth=(username, password))

In [4]:
result_df = pd.read_excel('result_df_with_sentiment.xlsx')

This part of work processes company data, creating a Neo4j graph database. It establishes relationships between companies based on data from CSV files, including both one-way and two-way connections. Additionally, it tracks and reports the total execution time.

In [10]:
# Read company information and create a mapping from ID to stock code
df_companies = pd.read_csv('KnowledgeGraph/hidy.nodes.company.csv')
id_to_code = dict(zip(df_companies[':ID'], df_companies['code']))

# Start timing
start_time = time.time()

# Create Neo4j graph database
with driver.session() as session:
    # Create company nodes
    for index, row in df_companies.iterrows():
        session.run("MERGE (:Company {name: $company_name, code: $code})", 
                    company_name=row['company_name'], code=row['code'])

    # Handle unidirectional relationship types
    for rel in ['invest', 'supply']:
        df_rel = pd.read_csv(f'KnowledgeGraph/hidy.relationships.{rel}.csv')
        for index, row in df_rel.iterrows():
            start_code = id_to_code.get(row[':START_ID'])
            end_code = id_to_code.get(row[':END_ID'])
            if start_code and end_code:
                session.run(f"MATCH (a:Company {{code: $start_code}}), (b:Company {{code: $end_code}}) "
                            f"MERGE (a)-[:{rel.upper()}]->(b)",
                            start_code=start_code, end_code=end_code)

    # Handle bidirectional relationship types
    for rel in ['compete', 'cooperate', 'dispute', 'same_industry']:
        df_rel = pd.read_csv(f'KnowledgeGraph/hidy.relationships.{rel}.csv')
        for index, row in df_rel.iterrows():
            start_code = id_to_code.get(row[':START_ID'])
            end_code = id_to_code.get(row[':END_ID'])
            if start_code and end_code:
                session.run(f"MATCH (a:Company {{code: $start_code}}), (b:Company {{code: $end_code}}) "
                            f"MERGE (a)-[:{rel.upper()}]->(b) "
                            f"MERGE (b)-[:{rel.upper()}]->(a)",
                            start_code=start_code, end_code=end_code)

# Close the database connection driver
driver.close()

# Total running time
total_run_time = time.time()
print("Total run time: {:.2f} seconds".format(total_run_time - start_time))


C:\Users\84166\AppData\Local\Temp\ipykernel_16420\3852314394.py:9: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
Failed to write data to connection IPv4Address(('localhost', 7687)) (ResolvedIPv4Address(('127.0.0.1', 7687)))


Total run time: 40.13 seconds
